## 패키지 임포트

In [ ]:
import pandas as pd
import random
import os

import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

import scipy
from scipy.stats import chisquare #EDA에서 사용한 모듈


## 데이터 입출력 경로

In [ ]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

## 랜덤시드 고정

In [ ]:
#baseline과 동일합니다. 
class CFG:
    SEED = 42
def seed_everything(seed):
    random.seed(seed) 
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED)



## 데이터 EDA
- 데이터 EDA를 통해 알고자 한 것은 다음과 같습니다.
- "어떤 변수가 각 종을 분류하는데에 결정적인 역할을 하는가?"
- 이를 다음 두 가지 질문으로 세분화했습니다. 
    1. 종에 따라, 각 변수의 비율이 어떻게 다른가? 
    2. 각 종에서, 어떤 SNP끼리 독립이 아닌가? 
      
- 1번 질문에 답하고자, 종별로 나누어서 각 변수의 도수분포표를 구했습니다. 이를 통해, B, C그룹이 유전적으로 가깝다는 것을 확인했습니다. 또한, 동질성 검정을 했습니다. 그 결과, 모든 SNP에서 그룹(A, B, C) 간 차이가 있다는 것을 확인했습니다.  
- 2번 질문에 답하고자, 그룹 별로 나누어서 독립성 검정을 했습니다. 그 결과, 각 그룹에서 독립이 아닌 SNP 쌍을 찾아낼 수 있었습니다. 
- 이 중, 각 그룹에서 독립인 SNP쌍이 있고 아닌 SNP쌍이 있다는 점에 집중해서 추가 변수를 생성했습니다. 
- 최종 코드에서 사용한 것은 독립성 검정 결과이므로 이 코드만 첨부합니다. 

In [ ]:
#종별로 데이터셋 분할.
groups=train.drop(columns=["id","father","mother","gender"]).groupby(train["class"])

train_x_A=groups.get_group("A") 
train_x_B=groups.get_group("B") 
train_x_C=groups.get_group("C") 


In [ ]:
#독립성 검정 함수를 만들어서 각 데이터셋에 적용. 
def independence_test(rawdata):
    dataset=rawdata.drop(columns=["trait","class"])
    snp_list=dataset.columns
    h0_snp=[]
    for snp_i1 in range(15):
        snp1=snp_list[snp_i1]
        for snp_i2 in range(14)[snp_i1+1:]:

            snp2=snp_list[snp_i2]
            b=dataset[[snp1,snp2]]
            print(snp1,snp2)
            
            #관측도수 구하기
            O=pd.crosstab(b[snp1],b[snp2])

            #변수명
            group=O.columns
            acid=O.index

            #기대도수 구하기
            class_sum=O.sum()
            c=list(class_sum)
            full_sum=class_sum.sum()
            snp_sum=O.sum(axis=1)
            s=list(snp_sum)

            E_=np.outer(s, c/full_sum, out=None)
            E=pd.DataFrame(E_,index=acid,columns=group)
            result = chisquare(O, f_exp=E,axis=None)
            print(f"H0: {snp1}과 {snp2} 간 상관관계가 없다. ")
            if result.pvalue<=0.025:
                
                print("유의확률은",result.pvalue,"로 귀무가설을 기각했습니다.")
                h0_snp.append([snp1,snp2])

    print(h0_snp)

In [ ]:
#독립성 검정(A)
#independence_test(train_x_A)

In [ ]:
#독립성 검정(B)
#independence_test(train_x_B)

In [ ]:
#독립성 검정(C)
#independence_test(train_x_C)

**결론은 다음과 같습니다.**

* A에서는 [['SNP_04', 'SNP_09'], ['SNP_05', 'SNP_06'], ['SNP_05', 'SNP_07'], ['SNP_06', 'SNP_07']]가 독립이 아닌 쌍이다. 
* B에서는 [['SNP_08', 'SNP_09']]가 독립이 아닌 쌍이다. 
* C에서는 [['SNP_03', 'SNP_05'], ['SNP_05', 'SNP_08'], ['SNP_06', 'SNP_08'], ['SNP_06', 'SNP_10'], ['SNP_07', 'SNP_08']]가 독립이 아닌 쌍이다.
  
 

**보다 엄격한 검정을 위해 p-value는 0.025로 설정했습니다.**

## 데이터 전처리(1)
- 추가 변수 생성 
    - EDA를 통해 각 그룹(A, B, C)에서 독립이 아닌 쌍을 모두 찾았습니다. 
    - 찾은 쌍들의 염기를 단순하게 이어붙여서, 새로운 변수로 추가했습니다. 
    - 예) SNP_8과 SNP_9가 A에서 독립이 아닌 쌍이라는 결론이 났다면->snp_8_9에는 SNP_8과 SNP_9의 값을 합친 문자열이 들어갑니다. 
    -(예: SNP_8이 "AA", SNP_9가 "GG"라면 snp_8_9는 "AA GG")
- ["id","father","mother","gender"] 변수 제거
    - 의미없는 변수이므로 제거합니다.

In [ ]:
def preprocessing_(data):
    #변수 제거
    data_drop=data.drop(columns=["id","father","mother","gender"])
    
    #추가변수 생성
    
    new_var_list=["snp_4_9", "snp_5_6","snp_5_7", "snp_8_9", "snp_3_5", "snp_5_8", "snp_6_8", "snp_6_10","snp_7_8"]
    ##new_var_list: 새로 만들 변수의 이름을 넣은 리스트. 
    
    for new_var in new_var_list:
        data_drop[new_var]=["Z"]*len(data_drop) #추가변수 값을 초기화. 

    ##추가변수 값 채워넣기. 
    data_drop["snp_4_9"]=data_drop["SNP_04"]+data_drop["SNP_09"]
    data_drop["snp_5_6"]=data_drop["SNP_05"]+data_drop["SNP_06"]
    data_drop["snp_5_7"]=data_drop["SNP_05"]+data_drop["SNP_07"]
    data_drop["snp_8_9"]=data_drop["SNP_08"]+data_drop["SNP_09"]
    data_drop["snp_3_5"]=data_drop["SNP_03"]+data_drop["SNP_05"]
    data_drop["snp_5_8"]=data_drop["SNP_05"]+data_drop["SNP_08"]
    data_drop["snp_6_8"]=data_drop["SNP_06"]+data_drop["SNP_08"]
    data_drop["snp_6_10"]=data_drop["SNP_06"]+data_drop["SNP_10"]
    data_drop["snp_7_8"]=data_drop["SNP_07"]+data_drop["SNP_08"]
    data_drop["snp_3_5"]=data_drop["SNP_03"]+data_drop["SNP_05"]
    data_drop["snp_5_8"]=data_drop["SNP_05"]+data_drop["SNP_08"]
    data_drop["snp_6_8"]=data_drop["SNP_06"]+data_drop["SNP_08"]
    data_drop["snp_6_10"]=data_drop["SNP_06"]+data_drop["SNP_10"]
    data_drop["snp_7_8"]=data_drop["SNP_07"]+data_drop["SNP_08"]
    
    #train, test셋 나누기.
    if "class" in data.columns:
        df_x = data_drop.drop(columns=['class'])
        df_y = data_drop['class']
        return df_x, df_y
    else: #test 데이터에서 id를 drop하고 x 구하기
        df_x = data_drop
        return df_x

In [ ]:
train_x, train_y = preprocessing_(train) 
test_x = preprocessing_(test) 

## 데이터 전처리(2)
- 데이터 라벨링

In [ ]:
#SNP_01~SNP_15 라벨링
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)] 

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values) 

train_y = class_le.fit_transform(train_y) 
snp_le.fit(snp_data)

for col in train_x.columns:
    if col in snp_col: 
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

In [ ]:
#추가변수 라벨링
snp_le_pair = preprocessing.LabelEncoder()
snp_col_pair = list(train_x.columns[16:])


snp_data_pair = []
for col in snp_col_pair:
    snp_data_pair += list(train_x[col].values) 

snp_le_pair.fit(snp_data_pair) 

for col in train_x.columns:
    if col in snp_col_pair: 
        train_x[col] = snp_le_pair.transform(train_x[col])
        test_x[col] = snp_le_pair.transform(test_x[col])

## 모델 적합시키기
- 랜덤서치를 통해 모델의 하이퍼파라미터 조정도 하였으나, 데이터셋이 작아서 오히려 과적합이 일어났습니다.
- 그래서 아무런 조정을 하지 않은 랜덤 포레스트 모델을 사용했습니다.

In [ ]:
clf = RandomForestClassifier(random_state=CFG.SEED) 
clf.fit(train_x, train_y)

## 결과값 도출하기

In [ ]:
preds = clf.predict(test_x)
print('Done.')

In [ ]:
submit = pd.read_csv('./sample_submission.csv')

In [ ]:
submit['class'] = class_le.inverse_transform(preds)

In [ ]:
submit.to_csv("./result.csv",index=False)